<a href="https://colab.research.google.com/github/lustraka/Predict_Ames_House_Prices/blob/main/collab/Logs_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import dependencies
import requests
import ast
import json
import pandas as pd
import os
import pickle

In [2]:
urls = {
    "Metrics" : 'https://raw.githubusercontent.com/lustraka/Predict_Ames_House_Prices/main/logs/session0723/best_run_metrics.json',
    "Details" : 'https://raw.githubusercontent.com/lustraka/Predict_Ames_House_Prices/main/logs/session0723/best_run_details.txt',
    "Model" : 'https://github.com/lustraka/Predict_Ames_House_Prices/blob/main/logs/session0723/AutoMLcd63750fd36.zip?raw=true'
}

In [3]:
# Model details
details = requests.get(urls['Details']).text
details = ast.literal_eval(details)
# A pattern for extracting structure of the VotingEnsemble model
voting_ensemble = {k:ast.literal_eval(details['properties'][k]) for k in ['ensembled_algorithms', 'ensemble_weights']}
pd.DataFrame(voting_ensemble).sort_values(by='ensemble_weights', ascending=False)

,ensembled_algorithms,ensemble_weights
0,XGBoostRegressor,0.400000
2,XGBoostRegressor,0.266667
1,LightGBM,0.133333
4,ElasticNet,0.133333
3,SGD,0.066667


In [4]:
# Model metrics
metrics = requests.get(urls['Metrics'])
metrics.json()

{'explained_variance': 0.886052769069071,
 'mean_absolute_error': 15439.116472115917,
 'mean_absolute_percentage_error': 9.122770171936464,
 'median_absolute_error': 9654.516791774067,
 'normalized_mean_absolute_error': 0.021440239511339974,
 'normalized_median_absolute_error': 0.013407188990104244,
 'normalized_root_mean_squared_error': 0.037139291460024836,
 'normalized_root_mean_squared_log_error': 0.042947817229549566,
 'predicted_true': 'aml://artifactId/ExperimentRun/dcid.AutoML_cd63750f-df45-4803-b135-af0f23de09cd_36/predicted_true',
 'r2_score': 0.885898472424608,
 'residuals': 'aml://artifactId/ExperimentRun/dcid.AutoML_cd63750f-df45-4803-b135-af0f23de09cd_36/residuals',
 'root_mean_squared_error': 26744.00378036388,
 'root_mean_squared_log_error': 0.13203033398097183,
 'spearman_correlation': 0.9573852130501542}

# Working With a Model

In [5]:
import zipfile
import pandas as pd

In [6]:
model = requests.get(urls['Model'])
with open('model.zip', 'bw') as archive:
  archive.write(model.content)
with zipfile.ZipFile('model.zip', 'r') as file:
  file.extractall()

In [7]:
os.listdir()

['.config',
 'model.zip',
 'model.pkl',
 'conda_env_v_1_0_0.yml',
 'scoring_file_v_1_0_0.py',
 'sample_data']

In [8]:
! pip install --upgrade azureml-sdk[notebooks,automl]

In [9]:
import pickle
with open('model.pkl', 'br') as p:
  model = pickle.load(p, encoding='utf-8')

Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception cannot import name 'RunType' from 'azureml.automl.core._run.types' (/usr/local/lib/python3.7/dist-packages/azureml/automl/core/_run/types.py).


In [10]:
type(model)

azureml.automl.runtime.shared.model_wrappers.RegressionPipeline

In [11]:
dir(model)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_estimator_type',
 '_final_estimator',
 '_fit',
 '_get_ci',
 '_get_param_names',
 '_get_params',
 '_get_tags',
 '_inverse_transform',
 '_iter',
 '_log_message',
 '_more_tags',
 '_pairwise',
 '_quantiles',
 '_replace_estimator',
 '_required_parameters',
 '_set_params',
 '_stddev',
 '_transform',
 '_validate_names',
 '_validate_steps',
 'classes_',
 'decision_function',
 'fit',
 'fit_predict',
 'fit_transform',
 'get_params',
 'inverse_transform',
 'memory',
 'named_steps',
 'pipeline',
 'predict',
 'predict_log_proba',
 'pr

In [13]:
params = model.get_params()
params

{'datatransformer': DataTransformer(
     task='regression',
     is_onnx_compatible=False,
     enable_feature_sweeping=True,
     enable_dnn=False,
     force_text_dnn=False,
     feature_sweeping_timeout=86400,
     featurization_config=None,
     is_cross_validation=True,
     feature_sweeping_config={}
 ),
 'datatransformer__enable_dnn': False,
 'datatransformer__enable_feature_sweeping': True,
 'datatransformer__feature_sweeping_config': {},
 'datatransformer__feature_sweeping_timeout': 86400,
 'datatransformer__featurization_config': None,
 'datatransformer__force_text_dnn': False,
 'datatransformer__is_cross_validation': True,
 'datatransformer__is_onnx_compatible': False,
 'datatransformer__observer': None,
 'datatransformer__task': 'regression',
 'datatransformer__working_dir': '/content',
 'pipeline': Pipeline(memory=None,
          steps=[('datatransformer',
                  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature

In [14]:
params.keys()

dict_keys(['pipeline__memory', 'pipeline__steps', 'pipeline__verbose', 'pipeline__datatransformer', 'pipeline__prefittedsoftvotingregressor', 'pipeline__datatransformer__task', 'pipeline__datatransformer__is_onnx_compatible', 'pipeline__datatransformer__enable_feature_sweeping', 'pipeline__datatransformer__enable_dnn', 'pipeline__datatransformer__force_text_dnn', 'pipeline__datatransformer__feature_sweeping_timeout', 'pipeline__datatransformer__featurization_config', 'pipeline__datatransformer__is_cross_validation', 'pipeline__datatransformer__feature_sweeping_config', 'pipeline__datatransformer__observer', 'pipeline__datatransformer__working_dir', 'pipeline__prefittedsoftvotingregressor__estimators', 'pipeline__prefittedsoftvotingregressor__weights', 'pipeline', 'stddev', 'datatransformer', 'prefittedsoftvotingregressor', 'datatransformer__task', 'datatransformer__is_onnx_compatible', 'datatransformer__enable_feature_sweeping', 'datatransformer__enable_dnn', 'datatransformer__force_te